# Model Creation

In this jupyter notebook we shall look at taking the preprocessed data  generated by preprocessing_part_2.ipynb and creating machine learning model from it 
that reads each review and tries to predict what its average score is. Thus we are building a text classifier

In [2]:
#start with the relevant imports

#use to visualise the data 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#used to build the model
import tensorflow as tf
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import Sequential
from keras.layers import Dense, Softmax, Activation, TextVectorization, Dropout
from keras.optimizers import Adam
from keras.losses import SparseCategoricalCrossentropy

Step 1: load and inspect the csv with pandas

In [5]:
#first load the data with pandas
df=pd.read_csv("./data/data_ready_for_model.csv")


In [8]:
df.head()

,Unnamed: 0,Comments,Average Score
0,0,Moved back to the UK end of August and got Vir...,1.0
1,1,"A truly attrocious service, both in terms of b...",1.0
2,2,They make it as hard as they can for you to ca...,2.0
3,3,Pay for the 350Mbps package but only ever mana...,2.0
4,4,The worst customer service:\r-The bots ask irr...,2.0


In [10]:
df.drop("Unnamed: 0", axis=1, inplace=True) #unneeded column, resulted when csv was created from dataframe

Step 2 prepare the data into train val test sets (code is borrowed from my Wine reviews classification Neural Network)

In [ ]:
#split the data into train val test split
train, val, test = np.split(df.sample(frac=1), [int(0.8*len(df)), int(0.9 * len(df))])

In [ ]:
#creating a function that feeds data to the model

def df_to_dataset(dataframe, shuffle=True, batch_size=1024):
  df = dataframe.copy()
  labels = df.pop('Average Score')
  df = df["Comments"]
  ds = tf.data.Dataset.from_tensor_slices((df, labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  ds = ds.prefetch(tf.data.AUTOTUNE)
  return ds

In [ ]:
#creating instances of our train, val test data
train_data = df_to_dataset(train)
valid_data = df_to_dataset(val)
test_data = df_to_dataset(test)

# Training an LSTM model

In [ ]:
#max tokens=max no of words we will remember.
encoder=TextVectorization(max_tokens=2000)
#train data is composed of comment and the score but we don't really 
# #need the score for this encoder. So just use a lambda function to pass in the text.
encoder.adapt(train_data.map(lambda comment, score: comment))